In [41]:
!pip install rank_bm25a

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement rank_bm25a (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for rank_bm25a


In [ ]:
!pip install boto3

In [1]:
import os
import json
import boto3
import random
from time import time
from datetime import datetime
from typing import List, Dict, Any
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.messages import AIMessage

In [58]:

load_dotenv()

class Config:
    AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    API_VERSION = "2024-02-01"
    PERSIST_DIRECTORY = "askhr_bot_vectorstore"
    COLLECTION_NAME = "askhr_bot_vectorstore_collection"
    DYNAMODB_TABLE_NAME = os.getenv("DYNAMODB_TABLE_NAME")

In [3]:
embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=Config.AZURE_OPENAI_ENDPOINT,
    api_key=Config.AZURE_OPENAI_API_KEY,
    openai_api_version=Config.API_VERSION
)


In [4]:
llm = AzureChatOpenAI(
    api_key=Config.AZURE_OPENAI_API_KEY,
    azure_endpoint=Config.AZURE_OPENAI_ENDPOINT,
    api_version=Config.API_VERSION,
    deployment_name=Config.AZURE_OPENAI_DEPLOYMENT_NAME,
    temperature=0.5,
)

In [6]:
def chatWithLLM(prompt: str) -> str:
    try:
        response = llm.invoke(prompt)
        if isinstance(response, AIMessage):
            return response.content
        return response
    except Exception as e:
        print(f"Error in LLM invocation: {e}")
        return "I'm having trouble processing that request right now."

In [7]:
chatWithLLM("Who is Sreen narayana guru")  # Test LLM invocation

'Sree Narayana Guru (1856–1928) was a prominent social reformer, spiritual leader, and philosopher from Kerala, India. He is best known for his efforts to promote social equality and uplift marginalized communities, particularly the lower castes in the Hindu social hierarchy. \n\nGuru emphasized the importance of education, self-respect, and spiritual awakening. He advocated for a society free from caste discrimination and worked towards the empowerment of the oppressed. His famous motto, "One caste, one religion, one God for mankind," reflects his vision of unity and equality among all people.\n\nSree Narayana Guru founded several institutions, including schools and temples, that aimed to promote education and social reform. He also established the Sree Narayana Dharma Paripalana (SNDP) Yogam, an organization dedicated to the welfare of the backward communities in Kerala.\n\nHis teachings and philosophies continue to inspire many, and he is revered as a saint and a key figure in the s

In [5]:
vectorstore = Chroma(
    collection_name=Config.COLLECTION_NAME,
    embedding_function=embedding_model,
    persist_directory=Config.PERSIST_DIRECTORY,
    collection_metadata={"hnsw:space": "cosine"}
)

In [7]:
def initialize_retrievers():
    try:
        raw = vectorstore.get(include=["documents", "metadatas"])
        docs = [
            Document(page_content=content, metadata=metadata)
            for content, metadata in zip(raw["documents"], raw["metadatas"])
        ]

        bm25_retriever = BM25Retriever.from_documents(docs, k=5)

        vector_retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 7, "fetch_k": 20, "lambda_mult": 0.6, "score_threshold": 0.7}
        )

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.4, 0.6]
        )

        QUERY_PROMPT = PromptTemplate(
            input_variables=["question"],
            template="""You are an AI language model assistant. Your task is to generate five 
            different versions of the given user question to retrieve relevant documents from a 
            vector database. Provide these alternative questions separated by newlines.
            Original question: {question}"""
        )

        return MultiQueryRetriever.from_llm(
            retriever=ensemble_retriever,
            llm=llm,
            prompt=QUERY_PROMPT,
            include_original=True
        )

    except Exception as e:
        print(f"Error initializing retrievers: {e}. Falling back to simple retriever.")
        return vectorstore.as_retriever(search_kwargs={"k": 5})


In [8]:
retriever = initialize_retrievers()

In [59]:
from boto3.dynamodb.conditions import Key,Attr
dynamodb = boto3.resource('dynamodb',region_name='ap-south-1')
TABLE_NAME = Config.DYNAMODB_TABLE_NAME
table = dynamodb.Table(TABLE_NAME)

class ChatHistoryManager:
    def __init__(self, user_id: str = "default", session_id: str = "default_session"):
        self.user_id = user_id
        self.session_id = session_id

    def load_history(self) -> List[Dict[str, Any]]:
        today = datetime.now().date()
        start_time = datetime.combine(today, datetime.min.time()).isoformat()
        end_time = datetime.combine(today, datetime.max.time()).isoformat()

        try:
            response = table.query(
                KeyConditionExpression=Key('user_id').eq(self.user_id) & Key('timestamp').between(start_time, end_time),
                FilterExpression=Attr('session_id').eq(self.session_id)
            )
            return response.get('Items', [])
        except Exception as e:
            print("Error loading from DynamoDB:", e)
            return []

    def save_history(self, history: List[Dict[str, Any]]):
        for entry in history:
            item = {
                'user_id': self.user_id,
                'timestamp': entry.get("timestamp", datetime.now().isoformat()),  # sort key
                'session_id': entry.get("session_id", self.session_id),
                'user_message': entry.get("user_message", ""),
                'assistant_response': entry.get("assistant_response", ""),
                'summarized': entry.get("summarized", False)
            }

            if "summary" in entry:
                item["summary"] = entry["summary"]
            if "summary_of" in entry:
                item["summary_of"] = entry["summary_of"]

            try:
                table.put_item(Item=item)
            except Exception as e:
                print("Error writing to DynamoDB:", e)

    def summarize_if_needed(self, history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        unsummarized_blocks = [
            entry for entry in history
            if not entry.get("summarized", False)
            and entry.get("user_message") and entry.get("assistant_response")
        ]

        if len(unsummarized_blocks) < 5:
            return history

        history_text = "\n".join(
            f"User: {entry['user_message']}\nAssistant: {entry['assistant_response']}"
            for entry in unsummarized_blocks[:10]
        )

        summary_prompt = f"""
        Summarize the following 10 interactions into one concise but informative summary:
        {history_text}
        """

        summary = llm.invoke(summary_prompt.strip())
        if isinstance(summary, AIMessage):
            summary = summary.content

        summary_entry = {
            "role": "system",
            "summary": summary,
            "timestamp": datetime.now().isoformat(),
            "session_id": self.session_id,
            "summarized": True,
            "summary_of": [entry["timestamp"] for entry in unsummarized_blocks[:10]]
        }

        history = [entry for entry in history if entry not in unsummarized_blocks[:10]]
        history.insert(0, summary_entry)
        return history

    def append_chat_pair(self, history: List[Dict[str, Any]], user_msg: str, assistant_msg: str) -> List[Dict[str, Any]]:
        new_entry = {
            "user_message": user_msg,
            "assistant_response": assistant_msg,
            "timestamp": datetime.now().isoformat(),
            "session_id": self.session_id,
            "summarized": False
        }

        history.append(new_entry)
        self.save_history([new_entry])  # Save only the new one
        return self.summarize_if_needed(history)

In [14]:
class ResponseEvaluator:
    def __init__(self):
        self.evaluation_history = []
    
    def log_interaction(self, user_input, response, context, retrieval_time):
        self.evaluation_history.append({
            "timestamp": datetime.now().isoformat(),
            "input": user_input,
            "response": response,
            "context_relevance": self._calculate_context_relevance(response, context),
            "retrieval_time": retrieval_time
        })
        self.evaluation_history = self.evaluation_history[-100:]
        
    def _calculate_context_relevance(self, response, context):
        if not context:
            return 0
        context_keywords = set(" ".join(context).split())
        response_keywords = set(response.split())
        common = context_keywords & response_keywords
        return len(common) / len(context_keywords) if context_keywords else 0
    
    def get_metrics(self):
        if not self.evaluation_history:
            return {}
        avg_relevance = sum(
            e["context_relevance"] for e in self.evaluation_history
        ) / len(self.evaluation_history)
        avg_time = sum(
            e["retrieval_time"] for e in self.evaluation_history
        ) / len(self.evaluation_history)
        return {
            "avg_context_relevance": avg_relevance,
            "avg_response_time": avg_time,
            "total_interactions": len(self.evaluation_history)
        }

In [15]:
evaluator = ResponseEvaluator()

In [14]:
def get_dynamic_prompt(user_input: str, history: List) -> PromptTemplate:
    sensitive_keywords = ["complaint", "harassment", "grievance", "termination"]
    policy_keywords = ["policy", "rule", "guideline"]
    benefit_keywords = ["benefit", "pto", "leave", "insurance"]
    
    if any(kw in user_input.lower() for kw in sensitive_keywords):
        instructions = "This is a sensitive topic. Be professional and direct the user to official HR channels if appropriate."
    elif any(kw in user_input.lower() for kw in policy_keywords):
        instructions = "Provide exact policy details with reference to the policy document when possible."
    elif any(kw in user_input.lower() for kw in benefit_keywords):
        instructions = "Include eligibility requirements and any limitations for benefits mentioned."
    else:
        instructions = "Respond helpfully and professionally."
    
    template = f"""You are an HR assistant for a company. Use the following context to answer the question at the end.
If you don't know the answer, say you don't know. Be concise but helpful.

Context:
{{context}}

Conversation history:
{{chat_history}}

Question: {{input}}

Considerations:
1. {instructions}
2. Format lists and important details clearly
3. Provide sources when available

Answer:"""
    
    return PromptTemplate.from_template(template)


In [15]:
def docs_to_serializable(docs: List[Document]) -> List[Dict[str, Any]]:
    return [
        {
            "content": doc.page_content,
            "metadata": doc.metadata
        }
        for doc in docs
    ]

In [46]:
def chat(user_input: str, user_id: str = "default", session_id: str = "default_session") -> str:
    history_manager = ChatHistoryManager(user_id, session_id)
    chat_history = history_manager.load_history()

    formatted_history = []
    for entry in chat_history:
        if entry.get("summarized", False):
            formatted_history.append(("system", entry["summary"]))
        else:
            formatted_history.append(("user", entry["user_message"]))
            formatted_history.append(("assistant", entry["assistant_response"]))



    contextualize_prompt = ChatPromptTemplate.from_messages([
        ("system", "Given a chat history and the latest user question, formulate a standalone question."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])

    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_prompt)
    qa_prompt = get_dynamic_prompt(user_input, chat_history)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    start_time = time()
    try:
        response = rag_chain.invoke({
            "input": user_input,
            "chat_history": formatted_history
        })
        elapsed = time() - start_time

        # Make sure response["answer"] is a plain string
        answer = response["answer"]
        if isinstance(answer, AIMessage):
            answer = answer.content


        chat_history = history_manager.append_chat_pair(
            chat_history, user_msg=user_input, assistant_msg=answer
        )

        history_manager.save_history(chat_history)

        return answer

    except Exception as e:
        print(f"Error in RAG chain: {e}")
        fallback_responses = [
            f"I'm having trouble accessing that information. Could you rephrase your question? (Error: {str(e)[:50]})",
            "My knowledge base seems to be unavailable at the moment. Please try again later.",
            "I encountered an unexpected error while processing your request."
        ]
        return random.choice(fallback_responses)


In [50]:
chat("what do you know about the CEO of AyataCommerce?","123","1234")

"The CEO of AyataCommerce is Shine Mathew. Here are some key details about him:\n\n- **Role**: Founder and CEO of AyataCommerce.\n- **Vision**: He emphasizes the importance of a transparent, open, and respectful work culture at AyataCommerce.\n- **Leadership Style**: Shine promotes mutual trust and encourages employees to connect, communicate, and collaborate effectively.\n- **Company Philosophy**: He advocates for a work environment that balances professional commitments with personal well-being.\n\nFor more information about AyataCommerce and its values, please refer to the company's official communications or internal resources."

In [57]:
chat("what do you know about him?","123","1234")

'Shine Mathew is the CEO and Founder of AyataCommerce. Here are some key details about him:\n\n- **Role**: Founder and CEO of AyataCommerce.\n- **Vision**: He emphasizes the importance of a transparent, open, and respectful work culture.\n- **Leadership Style**: Promotes mutual trust and encourages effective communication and collaboration among employees.\n- **Company Philosophy**: Advocates for a balance between professional commitments and personal well-being, fostering a supportive environment for employees.\n\nFor further insights, you may refer to internal communications or company resources related to AyataCommerce.'

In [20]:
chat("What are the core values of the company?","123","1234")

"The core values of AyataCommerce are:\n\n1. **Empathy**: Emphasizing understanding and valuing diverse perspectives within the global team.\n2. **Trust**: Building a transparent and respectful environment that fosters mutual trust.\n3. **Adaptability**: Encouraging flexibility and openness to innovative ideas and changes.\n\nThese values guide the company's working practices and interactions with clients and each other."

In [ ]:
chat("Which among the values reflects about the flexibility?","123")

Error in RAG chain: Object of type AIMessage is not JSON serializable


'My knowledge base seems to be unavailable at the moment. Please try again later.'

In [61]:
chat("Is it ok to take a half day leave if i am sick suddenly?","123","1234")

"Yes, you can take a half-day leave if you are suddenly sick. Here are the key points to consider:\n\n### Eligibility Requirements:\n1. **Notification**: You must inform your manager as soon as possible on the first day of your absence.\n2. **Duration of Absence**: If you are going to be away from work for more than 48 hours, you must notify your manager and may need to provide a medical certificate if the absence extends beyond two days.\n\n### Limitations:\n- **Sick Pay**: Ensure you understand the rules regarding company sick pay. If you have any uncertainties, it's advisable to ask payroll for clarification.\n- **Documentation**: If your illness continues beyond the term of your medical certificate, further documentation may be required to confirm your sickness.\n\n### Additional Notes:\n- You should record your leave accurately in Zoho.\n- If you are sick during a holiday, you must notify your manager promptly to potentially treat it as sick leave rather than annual leave.\n\nFor 